In [15]:
# Python library imports: numpy, random, sklearn, pandas, etc

import warnings
warnings.filterwarnings('ignore')

import sys
import random
import numpy as np

# cross_validation is deprecated since version 0.18. This module will be removed in 0.20.
# Use sklearn.model_selection.train_test_split instead.
# from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Config steps

In [16]:
TRAINING_MATRIX_RESULT_FILE_NAME = 'training_matrix_result.pickle'

### Import training data

In [ ]:
# Don't know if it's used or not
class Configuration:
    PG_HOST = 'business-intelligent-db.cw1neqwhyrda.eu-central-1.rds.amazonaws.com'
    PG_USER = 'postgres'
    PG_PWD = '52eXzbKbqmY2x45f'

from psycopg2 import connect
import pandas.io.sql as psql

# connect to our postgres DB
conn = connect(host=Configuration.PG_HOST,
                user=Configuration.PG_USER,
                password=Configuration.PG_PWD,
                database="airline",
                port=5432)

df = psql.read_sql('''
                    select dep_delay, month, day_of_month, day_of_week, crs_dep_time::time, distance, dest
                    from flight
                    where dep_delay > 15
                    and year = 1995
                ''', conn)


In [33]:
# read files
cols = ['dep_delay', 'month', 'day_of_month', 'day_of_week', 'dest',  'distance']
col_types = {'dep_delay': int, 'month': int, 'day_of_month': int, 'day_of_week': int, 'distance': int, 
            'dest': str}

df_1995 = pd.read_csv('result/1995.csv')
df_1996 = pd.read_csv('result/1996.csv')

# df_1996 = pd.read_csv('result/1996.csv')
# df_1996 = df_1996[cols]

df_years = [df_1995, df_1996]

flight_df = pd.concat(df_years)
flight_df

,year,month,day_of_month,day_of_week,dep_time,crs_dep_time,arr_time,crs_arr_time,unique_carrier,flight_num,...,air_time,arr_delay,dep_delay,origin,dest,distance,taxi_in,taxi_out,cancelled,diverted
0,1995,1,6,5,657,645,952,937,UA,482,...,83.0,15.0,12.0,ORD,PHL,678.0,7,25,0,0
1,1995,1,7,6,648,645,938,937,UA,482,...,88.0,1.0,3.0,ORD,PHL,678.0,5,17,0,0
2,1995,1,8,7,649,645,932,937,UA,482,...,83.0,-5.0,4.0,ORD,PHL,678.0,3,17,0,0
3,1995,1,9,1,645,645,928,937,UA,482,...,84.0,-9.0,0.0,ORD,PHL,678.0,3,16,0,0
4,1995,1,10,2,645,645,931,937,UA,482,...,82.0,-6.0,0.0,ORD,PHL,678.0,6,18,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5209321,1996,12,6,5,1512,0,1624,0,AA,1885,...,106.0,-12.0,-2.0,ATL,DFW,732.0,12,14,0,0
5209322,1996,12,7,6,1514,0,1633,0,AA,1885,...,115.0,-3.0,0.0,ATL,DFW,732.0,11,13,0,0
5209323,1996,12,8,7,1510,0,1650,0,AA,1885,...,129.0,14.0,-4.0,ATL,DFW,732.0,10,21,0,0
5209324,1996,12,9,1,1513,0,1616,0,AA,1885,...,105.0,-20.0,-1.0,ATL,DFW,732.0,9,9,0,0


In [34]:
# Select columns to reduce the training matrix side and define dimension for prediction feature
cols = ['unique_carrier', 'month', 'flight_num', 'day_of_month', 'day_of_week', 'dest', 'origin', 'dep_time', 'distance', 'dep_delay']
flight_df_selc = flight_df[cols]


In [35]:
flight_df_selc['dep_delay'] = (flight_df_selc['dep_delay'] > 10) *1
categ_cols = ['unique_carrier', 'flight_num', 'dest', 'origin']

for item  in categ_cols:
    flight_df_selc[item] = flight_df_selc[item].astype('category').cat.codes + 1


In [37]:
# Divide test and train matrix
train, test, y_train, y_test = train_test_split(flight_df_selc.drop(['dep_delay'], axis=1), flight_df_selc["dep_delay"],
                                                random_state=10, test_size=1)

In [38]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(train["unique_carrier"].values.reshape(-1, 1))  

# enc.feature_indices_
airline_onehot = enc.transform(train["unique_carrier"].values.reshape(-1, 1)).toarray()
airline_onehot_test = enc.transform(test["unique_carrier"].values.reshape(-1, 1)).toarray()

In [39]:
air_oh_df = pd.DataFrame(airline_onehot, columns = ["A1","A2","A3","A4","A5","A6","A7",
                                                    "A8","A9","A10"])
air_oh_test_df = pd.DataFrame(airline_onehot_test, columns = ["A1","A2","A3","A4","A5","A6","A7",
                                                         "A8","A9","A10"])

In [40]:
train2 = pd.concat([train.reset_index(),air_oh_df.reset_index()],axis=1).drop(["index","month","day_of_month","day_of_week","unique_carrier","flight_num","dest","origin"],axis=1)
test2  = pd.concat([test.reset_index(),air_oh_test_df.reset_index()],axis=1).drop(["index","month","day_of_month","day_of_week","unique_carrier","flight_num","dest","origin"],axis=1)

test2

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10
0,859,745.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# showing the performance of a classification model for binary
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))
# Parameter Tuning
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
             "n_estimators": [200],
              "learning_rate": [0.05,0.1,0.16]}

In [42]:
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_rf.fit(train2, y_train)

RandomForestClassifier(n_estimators=50, n_jobs=-1)

### Save training matrix result model to file

In [43]:
result_location_path = f"./training_results/{TRAINING_MATRIX_RESULT_FILE_NAME}"

# save model
pickle.dump(clf_rf, open(result_location_path, "wb"))


### Evaluate on test set

In [30]:
# Evaluate on test set
pr = clf_rf.predict(test2)

# print results
cm = confusion_matrix(y_test, pr)
print("Confusion matrix")
print(pd.DataFrame(cm))
report_svm = precision_recall_fscore_support(list(y_test), list(pr), average='micro')
print("\nprecision = %0.2f, recall = %0.2f, F1 = %0.2f, accuracy = %0.2f\n" % \
        (report_svm[0], report_svm[1], report_svm[2], accuracy_score(list(y_test), list(pr))))

Confusion matrix
         0       1
0  3173110  114911
1   287689  595677

precision = 0.90, recall = 0.90, F1 = 0.90, accuracy = 0.90



In [31]:
y_test2 = pd.DataFrame({"y":y_test.reset_index(drop=True)})
predict2 = pd.DataFrame({"yhat":pr})
x_test2 = pd.DataFrame(test2)
result = x_test2.join(y_test2)
predicted_result = result.join(predict2)
predicted_result

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,y,yhat
0,859,745.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1114,802.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
2,1525,308.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
3,1859,226.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,1745,1235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171382,1330,628.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
4171383,1809,745.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4171384,1818,134.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4171385,2057,349.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0


In [32]:
predicted_result[predicted_result['y'] != predicted_result['yhat']]

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,y,yhat
23,1226,1829.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
25,1640,129.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
26,1014,946.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
33,1047,1489.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1
36,1823,169.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171339,935,151.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4171347,1251,2176.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0
4171348,1836,440.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0
4171358,1747,934.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0
